In [1]:
import pandas as pd
import requests

#para normalizar fechas
import datetime
#para obtener fechas
import time

In [2]:
# Obtenemos información a partir de la API

url = f'https://www.ngdc.noaa.gov/hazel/hazard-service/api/v1/tsunamis/events?country=USA&maxYear=2023&minYear=2010'
dt_usa_prev_Mar = pd.DataFrame(requests.get(url).json()['items'])

url = f'https://www.ngdc.noaa.gov/hazel/hazard-service/api/v1/tsunamis/events?country=JAPAN&maxYear=2023&minYear=2010'
dt_jap_prev_Mar = pd.DataFrame(requests.get(url).json()['items'])

url = f'https://www.ngdc.noaa.gov/hazel/hazard-service/api/v1/tsunamis/events?country=CHILE&maxYear=2023&minYear=2010'
dt_chi_prev_Mar = pd.DataFrame(requests.get(url).json()['items'])

In [3]:
def mar(df):
    
    df=df.fillna(0)
    df['time'] = pd.to_datetime(df[['year','month','day','hour','minute','second']],format='%Y/%m/%d %H:%M:%S')
    df['time'] =df['time'].apply(lambda x: x.strftime('%Y/%m/%d %H:%M:%S'))
    df.rename(columns={'eqMagnitude':'mag', 'eqDepth':'deepth', 'latitude':'lat', 'longitude':'lng',
    'locationName':'place', 'maxWaterHeight':'mxwatheight'}, inplace=True)


    df['mxwatheight']  = df.mxwatheight.astype('float64')
    df.mag=df.mag.astype('float64')
    df.lat=df.lng.astype('float64')
    df.lat=df.lat.astype('float64')
    df.deepth=df.deepth.astype('float64')

    # Filtramos el dataframe con los datos relevantes.
    df = df[['country','mxwatheight','place', 'time', 'mag', 'lng',
    'lat', 'deepth' ]]
    return (df)    

In [4]:
dt_usa_Mar=mar(dt_usa_prev_Mar)
dt_chi_Mar=mar(dt_chi_prev_Mar)
dt_jap_Mar=mar(dt_jap_prev_Mar)

<h1>Transform to csv and upload to remote mysql</h1>

In [11]:
print(dt_jap_Mar.size, dt_usa_Mar.size, dt_chi_Mar.size)

168 144 112


In [5]:
dt_chi_Mar.head()

,country,mxwatheight,place,time,mag,lng,lat,deepth
0,CHILE,0.14,CENTRAL CHILE,2010/03/11 14:39:43,6.9,-71.891,-71.891,11.0
1,CHILE,0.16,CENTRAL CHILE,2020/12/27 21:39:14,6.7,-74.990,-74.990,10.0
2,CHILE,0.47,EASTER ISLAND REGION,2014/10/09 02:14:31,7.0,-110.811,-110.811,17.0
3,CHILE,13.60,CENTRAL CHILE,2015/09/16 22:55:26,8.3,-71.674,-71.674,22.0
4,CHILE,0.55,CENTRAL CHILE,2015/11/11 01:54:00,6.9,-72.120,-72.120,33.0


In [6]:
dt_jap_Mar.head()

,country,mxwatheight,place,time,mag,lng,lat,deepth
0,JAPAN,0.10,OFF FUKUSHIMA PREFECTURE,2021/02/13 14:07:50,7.1,141.749,141.749,49.0
1,JAPAN,0.20,HOKKAIDO ISLAND,2012/03/14 09:08:35,6.9,144.944,144.944,12.0
2,JAPAN,0.10,"NOTO PENINSULA, JAPAN",2023/05/05 05:42:04,6.2,137.305,137.305,8.0
3,JAPAN,2.90,"NAGASAKI, JAPAN",2019/03/20 00:00:00,0.0,0.000,0.000,0.0
4,JAPAN,0.09,OFF EAST COAST OF HONSHU ISLAND,2011/07/10 00:57:10,7.0,143.264,143.264,23.0


In [7]:
dt_usa_Mar.head()

,country,mxwatheight,place,time,mag,lng,lat,deepth
0,USA,0.22,NORTHEAST U.S.A.,2018/05/15 00:00:00,0.0,0.000,0.000,0.0
1,USA,0.40,HAWAII,2018/05/04 22:32:37,6.9,-154.998,-154.998,2.0
2,USA,0.50,GULF OF MEXICO,2020/04/23 00:00:00,0.0,0.000,0.000,0.0
3,USA,1.80,NORTHWEST ATLANTIC OCEAN,2013/06/13 00:00:00,0.0,0.000,0.000,0.0
4,USA,0.10,"ATKA, ALEUTIAN ISLANDS, AK",2011/06/24 03:09:39,7.3,-171.836,-171.836,52.0


In [10]:
dt_jap_Mar.to_csv('/home/juanm/Documents/historical_mar_japan.csv')
dt_usa_Mar.to_csv('/home/juanm/Documents/historical_mar_usa.csv')
dt_chi_Mar.to_csv('/home/juanm/Documents/historical_mar_chile.csv')

<h1>Connect with remote mysql row by row</h1>